In [2]:
import sys
sys.path.append("/u/dssc/zenocosini/helm_suite/MCQA_Benchmark")
from metrics.utils import hidden_states_collapse
from metrics.query import DataFrameQuery
from common.tensor_storage import TensorStorage
from common.metadata_db import MetadataDB
from metrics.utils import  exact_match, angular_distance

#from sklearn.feature_selection import mutual_info_regression MISSIN?
from dadapy.data import Data

from pathlib  import Path

import numpy as np
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer

from common.metadata_db import MetadataDB
from common.utils import *
from pathlib import Path
import pickle



/u/dssc/zenocosini/miniconda3/envs/mcqa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Functions

In [4]:
def set_dataframes(db) -> pd.DataFrame:
    """
    Aggregate in a dataframe the hidden states of all instances
    ----------
    hidden_states: pd.DataFrame(num_instances, num_layers, model_dim)
    """
    df = pd.read_sql("SELECT * FROM metadata", db.conn)
    df["train_instances"] = df["train_instances"].astype(str)
    df.drop(columns=["id"],inplace = True)
    #import pdb; pdb.set_trace()
    df.drop_duplicates(subset = ["id_instance"],inplace = True, ignore_index = True) # why there are duplicates???
    return df

In [5]:
def tensor_retrieve(dict_query):
    query = DataFrameQuery(dict_query)
    hidden_states,logits, hidden_states_df= hidden_states_collapse(metadata_df,tensor_storage,query)
    return hidden_states,logits,hidden_states_df

In [6]:
def constructing_labels(label: str, hidden_states_df: pd.DataFrame, hidden_states: np.ndarray) -> np.ndarray:
    labels_literals = hidden_states_df[label].unique()
    labels_literals.sort()
    
    map_labels = {class_name: n for n,class_name in enumerate(labels_literals)}
    
    label_per_row = hidden_states_df[label].reset_index(drop=True)
    label_per_row = np.array([map_labels[class_name] for class_name in label_per_row])[:hidden_states.shape[0]]
    
    return label_per_row, map_labels

In [7]:
_PATH = Path("/orfeo/scratch/dssc/zenocosini/commonsenseqa_letter_result/")
result_path = Path(_PATH,"diego")
result_path.mkdir(exist_ok=True,parents=True)
metadata_db = MetadataDB(_PATH / "metadata.db")
metadata_df = set_dataframes(metadata_db)
tensor_storage = TensorStorage(Path(_PATH, "tensor_files"))



## Tensor Retrieval

### Base model

In [28]:
shot = 0

In [9]:
!ls /orfeo/scratch/dssc/zenocosini/mmlu_result/transposed_dataset/llama-7b-base/0

distances-0.npy   distances-31.npy	dist_indices-22.npy
distances-10.npy  distances-32.npy	dist_indices-23.npy
distances-11.npy  distances-3.npy	dist_indices-24.npy
distances-12.npy  distances-4.npy	dist_indices-25.npy
distances-13.npy  distances-5.npy	dist_indices-26.npy
distances-14.npy  distances-6.npy	dist_indices-27.npy
distances-15.npy  distances-7.npy	dist_indices-28.npy
distances-16.npy  distances-8.npy	dist_indices-29.npy
distances-17.npy  distances-9.npy	dist_indices-2.npy
distances-18.npy  distances-logits.npy	dist_indices-30.npy
distances-19.npy  dist_indices-0.npy	dist_indices-31.npy
distances-1.npy   dist_indices-10.npy	dist_indices-32.npy
distances-20.npy  dist_indices-11.npy	dist_indices-3.npy
distances-21.npy  dist_indices-12.npy	dist_indices-4.npy
distances-22.npy  dist_indices-13.npy	dist_indices-5.npy
distances-23.npy  dist_indices-14.npy	dist_indices-6.npy
distances-24.npy  dist_indices-15.npy	dist_indices-7.npy
distances-25.npy  dist_indices-16.npy	dist_indices-8.n

In [29]:
datasets = list(metadata_df["dataset"].unique())
dict_query = {"dataset":datasets, 
              "method":"last",
              "model_name":"meta-llama/Llama-2-7b-chat-hf",
              "train_instances": shot}
hidden_states,logits, hidden_states_df = tensor_retrieve(dict_query)

 Tensor retrieval took: 197.59465193748474



In [10]:
logits.shape

(1221, 1, 32000)

In [8]:
! mkdir -p /orfeo/scratch/dssc/zenocosini/mmlu_result/transposed_dataset/llama-7b-base

In [11]:
! ls /orfeo/scratch/dssc/zenocosini/mmlu_result/transposed_dataset/llama-7b-base

0


In [10]:
! mkdir -p /orfeo/scratch/dssc/zenocosini/mmlu_result/transposed_dataset/llama-7b-base/0

In [30]:
path = Path(_PATH,"transposed_dataset","llama-7b-chat",str(shot))
path.mkdir(exist_ok=True,parents=True)

In [31]:
labels, map_dict = constructing_labels("dataset",hidden_states_df, hidden_states)
np.save(Path(path,"subjects-labels.pkl"),labels)
with open(Path(path,"subjects-map"),"wb") as f:
    pickle.dump(map_dict,f)
labels, map_dict = constructing_labels("only_ref_pred",hidden_states_df, hidden_states)
with open(Path(path,"letter-map.pkl"),"wb") as f:
    pickle.dump(map_dict,f)
np.save(Path(result_path,"letter-base"),labels)

In [32]:
dict_nn_matrix = {}
dict_nn_matrix_l = {}

for layer in range(hidden_states.shape[1]):
    data = Data(hidden_states[:,layer,:])
    data.compute_distances(maxk=150)
    
    np.save(Path(path,f"distances-{layer}"), data.distances)
    np.save(Path(path,f"dist_indices-{layer}"), data.dist_indices)
    
 


/u/dssc/zenocosini/miniconda3/envs/mcqa/lib/python3.11/site-packages/dadapy/_utils/utils.py:152: UserWarning: There are points with neighbours at 0 distance, meaning the dataset probably has identical points.
This can cause problems in various routines.
We suggest to either perform smearing of distances using
remove_zero_dists()
or remove identical points using
remove_identical_points()).
  warnings.warn(


In [33]:
data_l = Data(logits[:,0,:])
data_l.compute_distances(maxk=150)
np.save(Path(path,f"distances-logits"), data_l.distances)
np.save(Path(path,f"dist_indices-logits"), data_l.dist_indices)

In [23]:
data.distances.shape

(14015, 151)

In [24]:
logits.shape

(14015, 1, 32000)

In [61]:
with open(Path(result_path,"llama-7b-base-5shot-dist-matrix.pkl"),"wb") as f:
    pickle.dump(dict_nn_matrix,f)

### Chat model

In [65]:
datasets = list(metadata_df["dataset"].unique())
dict_query = {"dataset":datasets, 
              "method":"last",
              "model_name":"meta-llama/Llama-2-7b-chat-hf",
              "train_instances": 5}
hidden_states, hidden_states_df = tensor_retrieve(dict_query)

 Tensor retrieval took: 161.6436107158661



In [66]:
np.save(Path(result_path,"llama-7b-chat-5shot"), hidden_states)

In [67]:
labels, map_labels = constructing_labels("dataset",hidden_states_df, hidden_states)
np.save(Path(result_path,"subjects-chat"),labels)
labels = constructing_labels("only_ref_pred",hidden_states_df, hidden_states)
np.save(Path(result_path,"letter-chat"),labels)

In [68]:
dict_nn_matrix = {}
for layer in [6,15,18,29,31]:
    data = Data(hidden_states[:,layer,:])
    data.compute_distances(maxk=150)
    dict_nn_matrix[layer] = (data.distances,data.dist_indices)

In [69]:
with open(Path(result_path,"llama-7b-chat-5shot-dist-matrix.pkl"),"wb") as f:
    pickle.dump(dict_nn_matrix,f)

## Overlap Correct Answer

In [5]:
exact_matches = metadata_df.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
metadata_df["exact_match"] = exact_matches
metadata_df_correct = metadata_df[metadata_df["method"]=="last"].copy()

#WHATCH OUT: some instance in medicine realated subject are repeated.
metadata_df_correct["id_instance"] = metadata_df_correct.apply(lambda r: r["id_instance"][:92]+r["id_instance"][-1], axis=1)
metadata_df_correct.drop_duplicates(subset=["id_instance"],inplace = True)
metadata_df_correct["id_instance"] = metadata_df_correct.apply(lambda r: r["id_instance"][:64], axis=1)

metadata_df_correct.reset_index(inplace=True)
metadata_df_correct.drop(columns=["index"],inplace=True)

#creating pivot table
metadata_df_correct['match_comb'] = metadata_df_correct.apply(lambda row: f"{row['model_name']}_{row['train_instances']}", axis=1)
pivot_df = metadata_df_correct.pivot( index='id_instance',columns='match_comb', values='exact_match')
pivot_df.reset_index(inplace=True)

NameError: name 'metadata_df' is not defined

In [108]:
metadata_df_correct["model_name"]

0              meta-llama/Llama-2-7b-hf
2              meta-llama/Llama-2-7b-hf
4              meta-llama/Llama-2-7b-hf
6              meta-llama/Llama-2-7b-hf
8              meta-llama/Llama-2-7b-hf
                      ...              
168170    meta-llama/Llama-2-7b-chat-hf
168172    meta-llama/Llama-2-7b-chat-hf
168174    meta-llama/Llama-2-7b-chat-hf
168176    meta-llama/Llama-2-7b-chat-hf
168178    meta-llama/Llama-2-7b-chat-hf
Name: model_name, Length: 83622, dtype: object

In [115]:
from tabulate import tabulate
cols = ["meta-llama/Llama-2-7b-hf_0","meta-llama/Llama-2-7b-hf_5","meta-llama/Llama-2-7b-chat-hf_0","meta-llama/Llama-2-7b-chat-hf_5"]
matrix = []
for col1 in cols:
    for col2 in cols:
        if col1==col2:
            perc = 1
        else:
            perc = (pivot_df[[col1,col2]].apply(lambda r: r[col1]==r[col2] and r[col1]==True, axis = 1).sum()/len(pivot_df))*100
        matrix.append(perc)
matrix = np.array(matrix).reshape([4,4])
print(tabulate(matrix, headers=cols,showindex=cols))

                                   meta-llama/Llama-2-7b-hf_0    meta-llama/Llama-2-7b-hf_5    meta-llama/Llama-2-7b-chat-hf_0    meta-llama/Llama-2-7b-chat-hf_5
-------------------------------  ----------------------------  ----------------------------  ---------------------------------  ---------------------------------
meta-llama/Llama-2-7b-hf_0                                1                            36.2                               31.8                               32.4
meta-llama/Llama-2-7b-hf_5                               36.2                           1                                 44.6                               47.2
meta-llama/Llama-2-7b-chat-hf_0                          31.8                          44.6                                1                                 54.8
meta-llama/Llama-2-7b-chat-hf_5                          32.4                          47.2                               54.8                                1


In [16]:
logits_i.squeeze(1)

(1221, 32000)

## Overlap logits

In [1]:
import sys
sys.path.append("/u/dssc/zenocosini/helm_suite/MCQA_Benchmark")


In [2]:
from metrics.utils import hidden_states_collapse
from metrics.query import DataFrameQuery
from common.tensor_storage import TensorStorage
from common.metadata_db import MetadataDB
from metrics.utils import  exact_match, angular_distance

#from sklearn.feature_selection import mutual_info_regression MISSIN?
from dadapy.data import Data

from pathlib  import Path

import numpy as np
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer

from common.metadata_db import MetadataDB
from common.utils import *
from pathlib import Path
import pickle

/u/dssc/zenocosini/miniconda3/envs/mcqa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
_PATH = Path("/orfeo/scratch/dssc/zenocosini/mmlu_result/")
result_path = Path(_PATH,"diego")
result_path.mkdir(exist_ok=True,parents=True)
metadata_db = MetadataDB(_PATH / "metadata.db")
metadata_df = set_dataframes(metadata_db)
tensor_storage = TensorStorage(Path(_PATH, "tensor_files"))



In [9]:
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b")
index_letter = [tokenizer.encode(letter)[1] for letter in ["A","B","C","D"]]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
def angular_distance(mat):
    """
    Computes distance based on angles between vectors, over the rows of the matrix
    """
    dot_product = mat @ mat.T
    norm_vector = np.linalg.norm(mat, axis=1)
    stacked_vector = np.tile(norm_vector, (mat.shape[0], 1))
    norm_product = stacked_vector.T*stacked_vector

    cosine_similarity = dot_product / norm_product
    cosine_similarity = np.clip(cosine_similarity, -1, 1)
    distances = np.arccos(cosine_similarity) / np.pi
    #distances = np.arccos(cosine_similarity) / np.pi
    #distances.sort(axis=1)
    return distances

In [37]:
def overlap_computer(logits_1,logits_2, cosine = "False", k=30):
    if cosine:
        distances_1 = angular_distance(logits_1)
        distances_2 = angular_distance(logits_2)
        data = Data(distances=distances_1, maxk=600)
        #data.compute_neigh_indices()
        overlap = data.return_data_overlap(distances=distances_2, k=k)
    else:
        data = Data(logits_1,maxk=100)
        overlap = data.return_data_overlap(logits_2, k=k)
    print(f'{overlap=}')
    return overlap

In [12]:
def softmax(x):
    exp_values = np.exp(x)# - np.max(x, axis=-1, keepdims=True))
    probabilities = exp_values / np.sum(exp_values, axis=-1, keepdims=True)
    return probabilities

Setup

In [14]:
def setup(dict_query_i, dict_query_j):

    hidden_states_i, logits_i, df_i = tensor_retrieve(dict_query_i)
    hidden_states_j,logits_j, df_j = tensor_retrieve(dict_query_j)
    df_i["exact_match"] = df_i.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
    df_j["exact_match"] = df_j.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
    
    df_i.reset_index(inplace=True)
    df_j.reset_index(inplace=True)
    df_i.drop("index", axis=1,inplace=True)
    df_j.drop("index", axis=1,inplace=True)
    # find the index of rows that have "exact_match" True in both df_i and df_j
    indices_i = df_i[df_i["exact_match"] == True].index
    indices_j = df_j[df_j["exact_match"] == True].index
    
    # find the intersection of the two sets of indices
    indices = indices_i.intersection(indices_j)
    logits_i = logits_i.squeeze(1)
    logits_j = logits_j.squeeze(1)
    logits_i = logits_i[indices]
    logits_j = logits_j[indices]
    logits_i_only_letter = logits_i[:,index_letter]
    logits_j_only_letter = logits_j[:,index_letter]
    return ((logits_i, logits_j), (logits_i_only_letter, logits_j_only_letter))

In [10]:
def logits_overlap(dict_query_i,dict_query_j):
    standard_logits, letter_logits = setup(dict_query_i,dict_query_j)
    dict_result = {}
    logits_i,logits_j = standard_logits
    #softmax cosine   
    softmx_i = softmax(logits_i)
    softmx_j = softmax(logits_j)
    print("Computing softmax cosine")
    dict_result["softmax-cosine"] =  overlap_computer(softmx_i,softmx_j, cosine=True, k=500)
    #softmx_norm
    print("Computing softmax norm")
    logits_i_norm = logits_i/np.linalg.norm(logits_i,axis=1, ord=1, keepdims=True)  
    logits_j_norm = logits_j/np.linalg.norm(logits_j,axis=1, ord=1, keepdims=True)
    softmx_i_norm = softmax(logits_i_norm)
    softmx_j_norm = softmax(logits_j_norm)
    dict_result["softmax-norm"] =  overlap_computer(softmx_i_norm,softmx_j_norm, cosine=False, k=500)
    #softmax standard
    print("Computing softmax standard")
    dict_result["softmax-std"] =  overlap_computer(softmx_i,softmx_j, cosine=False, k=500)
    #standard
    print("Computing standard")
    dict_result["std"] =  overlap_computer(logits_i,logits_j, cosine=False, k=500)
    return dict_result
    

In [15]:
dict_query_i = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-hf",
              "train_instances": 5}
dict_query_j = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-hf",
              "train_instances": 0}
standard_logits, letter_logits = setup(dict_query_i,dict_query_j)

 Tensor retrieval took: 11.553614139556885

 Tensor retrieval took: 8.045417070388794



/tmp/ipykernel_1528525/2795238491.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i["exact_match"] = df_i.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_1528525/2795238491.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_j["exact_match"] = df_j.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_1528525/2795238491.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [16]:
logits_i,logits_j = standard_logits

In [17]:
overlap_computer(logits_i,logits_j, cosine=True, k=500)

/orfeo/cephfs/home/dssc/zenocosini/helm_suite/DADApy/dadapy/kstar.py:101: UserWarning: Careful! The intrinsic dimension is not defined. Computing it unsupervisedly with 'compute_id_2NN()' method
  warnings.warn(


overlap=0.28964559532876366


0.28964559532876366

In [18]:
logits_i_norm = logits_i/np.linalg.norm(logits_i,axis=1, ord=1, keepdims=True)  
logits_j_norm = logits_j/np.linalg.norm(logits_j,axis=1, ord=1, keepdims=True)

In [ ]:
overlap_norm = overlap_computer(logits_i_norm,logits_j_norm, cosine=False, k=500)

In [21]:
overlap_norm

0.286995684183803

In [25]:
dict_query_i = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-hf",
              "train_instances": 5}
dict_query_j = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-chat-hf",
              "train_instances": 0}
standard_logits, letter_logits = setup(dict_query_i,dict_query_j)

 Tensor retrieval took: 10.462408542633057

 Tensor retrieval took: 8.450390100479126



/tmp/ipykernel_1528525/2795238491.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i["exact_match"] = df_i.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_1528525/2795238491.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_j["exact_match"] = df_j.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_1528525/2795238491.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [44]:
logits_i,logits_j = standard_logits
overlap_computer(logits_i,logits_j, cosine=True, k=500)

overlap=0.34917683465959326


0.34917683465959326

In [ ]:
logits_i_norm = logits_i/np.linalg.norm(logits_i,axis=1, ord=2, keepdims=True)  
logits_j_norm = logits_j/np.linalg.norm(logits_j,axis=1, ord=2, keepdims=True)
overlap_norm = overlap_computer(logits_i_norm,logits_j_norm, cosine=False, k=500)

In [47]:
overlap_norm

0.3491746242263484

In [ ]:
overlap = overlap_computer(logits_i,logits_j, cosine=False, k=500)

In [48]:
overlap

0.3188549955791335

In [15]:
dict_query_i = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-chat-hf",
              "train_instances": 5}
dict_query_j = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-chat-hf",
              "train_instances": 0}
result_3 = logits_overlap(dict_query_i, dict_query_j)

 Tensor retrieval took: 201.71871066093445

 Tensor retrieval took: 8.206757307052612



/tmp/ipykernel_1430747/2795238491.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i["exact_match"] = df_i.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_1430747/2795238491.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_j["exact_match"] = df_j.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_1430747/2795238491.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

Computing standard
overlap=0.39740898345153663
Computing softmax standard
overlap=0.5114031642116749
Computing softmax cosine
overlap=0.5367288597926896
Computing softmax norm
overlap=0.4791936715766503


In [17]:
dict_query_i = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-hf",
              "train_instances": 5}
dict_query_j = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-hf",
              "train_instances": 2}
result_1 = logits_overlap(dict_query_i, dict_query_j)

 Tensor retrieval took: 9.75747561454773

 Tensor retrieval took: 256.61804580688477



/tmp/ipykernel_1430747/2795238491.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i["exact_match"] = df_i.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_1430747/2795238491.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_j["exact_match"] = df_j.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_1430747/2795238491.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

Computing standard
overlap=0.5884975462438655
Computing softmax standard
overlap=0.6475130237825595
Computing softmax cosine
overlap=0.6504326160815402
Computing softmax norm
overlap=0.6430403926009814


COSINE

In [198]:
softmx_i = softmax(logits_i)
softmx_j = softmax(logits_j)
softmx_i_norm = softmax(logits_i_norm)
softmx_j_norm = softmax(logits_j_norm)

In [199]:
overlap_computer(softmx_i,softmx_j, cosine=True, k=500)

overlap=0.5096862147753237


0.5096862147753237

In [200]:
overlap_computer(logits_i,logits_j, cosine=False, k=500)

overlap=0.27882305153592285


0.27882305153592285

In [201]:
overlap_computer(softmx_i,softmx_j, cosine=False, k=500)

overlap=0.4969256156384869


0.4969256156384869

In [202]:
overlap_computer(softmx_i_norm,softmx_j_norm, cosine=False, k=500)

overlap=0.35662553947702463


0.35662553947702463

Test

In [163]:
mx_index_i = np.argmax(logits_i,axis=1)
mx_index_j = np.argmax(logits_j,axis=1)

In [165]:
argmx_i = np.zeros([4524,3200])
argmx_i[mx_index_i] = 1
argmx_j = np.zeros([4524,3200])
argmx_j[mx_index_j] = 1

In [172]:
np.linalg.norm(softmx_i[0]-softmx_j[0])

0.37634236

In [179]:
softmx_i.shape

(4524, 32000)

In [181]:
data = Data(softmx_i,maxk=1000)
overlap = data.return_data_overlap(softmx_j, k=500)

In [182]:
overlap

0.48004288240495135

### Test

In [183]:
dict_query_i = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-hf",
              "train_instances": 5}
dict_query_j = {"method":"last",
              "model_name":"meta-llama/Llama-2-7b-chat-hf",
              "train_instances": 0}
standard_logits, letter_logits = setup(dict_query_i, dict_query_j)

 Tensor retrieval took: 84.841481924057

 Tensor retrieval took: 245.30576395988464



/tmp/ipykernel_3488545/2795238491.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i["exact_match"] = df_i.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_3488545/2795238491.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_j["exact_match"] = df_j.apply(lambda r: exact_match(r["std_pred"], r["letter_gold"]), axis=1)
/tmp/ipykernel_3488545/2795238491.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [187]:
logits_i,logits_j = standard_logits
logits_i_norm = logits_i/np.linalg.norm(logits_i,axis=1, ord=2, keepdims=True)  
logits_j_norm = logits_j/np.linalg.norm(logits_j,axis=1, ord=2, keepdims=True)

In [110]:
data_in = Data(logits_i_norm*1000)
dist_in = data_in.compute_distances()

In [111]:
dist_ind_in = data_in.dist_indices

In [112]:
dist_i = angular_distance(logits_i)
data_i = Data(distances=dist_i)
dist_i = data_i.compute_neigh_indices()
dist_ind_i = data_i.dist_indices

/orfeo/cephfs/home/dssc/zenocosini/helm_suite/DADApy/dadapy/kstar.py:101: UserWarning: Careful! The intrinsic dimension is not defined. Computing it unsupervisedly with 'compute_id_2NN()' method
  warnings.warn(


In [120]:
#test = logits_i[0]/np.linalg.norm(logits_i[0], ord=1)
np.allclose(dist_ind_i[:,1], dist_ind_in[:,1])

False

In [134]:
np.nonzero(dist_ind_i[:,90] - dist_ind_in[:,90])[0].shape

(54,)

In [153]:
sfmx_i

array([[4.10e-12, 2.09e-13, 1.55e-05, ..., 2.38e-11, 2.18e-11, 3.30e-11],
       [1.34e-11, 2.06e-12, 5.63e-06, ..., 2.19e-11, 1.18e-11, 2.60e-11],
       [3.45e-12, 1.51e-13, 1.13e-05, ..., 2.06e-11, 1.85e-11, 2.60e-11],
       ...,
       [3.02e-13, 1.16e-13, 2.60e-06, ..., 2.02e-12, 8.45e-12, 8.30e-12],
       [1.56e-12, 2.63e-13, 5.00e-06, ..., 3.22e-12, 1.55e-11, 9.66e-12],
       [3.13e-12, 1.08e-13, 2.64e-06, ..., 1.21e-11, 4.33e-11, 1.65e-11]],
      dtype=float32)

In [194]:
sfmx_i = softmax(logits_i)
sfmx_j = softmax(logits_j)
dist_i = angular_distance(logits_i)
dist_j = angular_distance(logits_j)
data_sm = Data(distances=dist_i)
overlap = data_sm.return_data_overlap(distances=dist_j, k=500)

/orfeo/cephfs/home/dssc/zenocosini/helm_suite/DADApy/dadapy/metric_comparisons.py:732: UserWarning: Chosen k = 500 is greater than max available number of                    nearest neighbors = 100. Setting k = 100
  dist_indices_base, k_base = self._get_nn_indices(


In [195]:
overlap

0.25827365163572064

In [193]:
sfmx_i = softmax(logits_i)
sfmx_j = softmax(logits_j)
dist_i = angular_distance(logits_i)
dist_j = angular_distance(logits_j)
data_sm = Data(logits_i_norm)
overlap = data_sm.return_data_overlap(logits_j_norm, k=500)
overlap

KeyboardInterrupt: 

Rotation

In [44]:
def rotate_vector(vector, angle_degrees):
    """
    Rotate a 2D vector by a given angle in degrees.

    Parameters:
    vector (tuple/list/np.array): The vector to rotate, represented as (x, y).
    angle_degrees (float): The rotation angle in degrees.

    Returns:
    np.array: The rotated vector.
    """
    # Convert angle from degrees to radians
    angle_radians = np.radians(angle_degrees)
    
    # Rotation matrix
    rotation_matrix = np.array([
        [np.cos(angle_radians), -np.sin(angle_radians)],
        [np.sin(angle_radians), np.cos(angle_radians)]
    ])
    
    Rotate the vector
    rotated_vector = np.dot(rotation_matrix, vector)
    
    return rotated_vector

In [196]:
v1[0,0] = 1
v1[1] = rotate_vector(v1[0],1)

In [197]:
angular_distance(v1)

array([[0.  , 0.01],
       [0.01, 0.  ]])

In [198]:
v1_norm = v1/np.linalg.norm(v1, axis=1, ord=1,keepdims=True)

In [199]:
v1_norm

array([[1.  , 0.  ],
       [0.98, 0.02]])